In [5]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Données synthétiques
X, y = make_regression(n_samples=500, n_features=6, noise=0.1, random_state=42)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === MODELE 1 (3 features) ===
X_train_old = X_train[:, :3]
X_test_old = X_test[:, :3]

model1 = Sequential([
    Input(shape=(3,), name='input_3f'),
    Dense(16, activation='relu', name='dense_1'),
    Dense(8, activation='relu', name='dense_2'),
    Dense(1, name='output')
])
model1.compile(optimizer='adam', loss='mse')

history1 = model1.fit(X_train_old, y_train, epochs=30, batch_size=16, validation_split=0.2, verbose=0)


In [3]:
# Sauvegarde du modèle complet
model1.save("model1.keras")

In [4]:
# Courbes d'apprentissage modèle 1
plt.figure(figsize=(10, 5))
plt.plot(history1.history['loss'], label='train_loss')
plt.plot(history1.history['val_loss'], label='val_loss')
plt.title("Modèle 1 - 3 features")
plt.xlabel("Époques")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True)
plt.savefig("loss_model1.jpg")
plt.close()

In [6]:
# === MODELE 2 (6 features) ===
model2 = Sequential([
    Input(shape=(6,), name='input_6f'),
    Dense(16, activation='relu', name='dense_1'),
    Dense(8, activation='relu', name='dense_2'),
    Dense(1, name='output')
])

In [7]:
# Charger le modèle 1 et transférer les poids compatibles
model1_loaded = tf.keras.models.load_model("model1.keras")

for layer in model2.layers:
    try:
        old_layer = model1_loaded.get_layer(layer.name)
        layer.set_weights(old_layer.get_weights())
        print(f"✅ Poids transférés pour : {layer.name}")
    except ValueError:
        print(f"⛔ Incompatible ou nouvelle couche : {layer.name}")

model2.compile(optimizer='adam', loss='mse')

⛔ Incompatible ou nouvelle couche : dense_1
✅ Poids transférés pour : dense_2
✅ Poids transférés pour : output


In [8]:
# Entraînement du modèle avec les 6 features
history2 = model2.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.2, verbose=0)



In [9]:
# Courbes d'apprentissage modèle 2
plt.figure(figsize=(10, 5))
plt.plot(history2.history['loss'], label='train_loss')
plt.plot(history2.history['val_loss'], label='val_loss')
plt.title("Modèle 2 - 6 features (avec transfert)")
plt.xlabel("Époques")
plt.ylabel("MSE Loss")
plt.legend()
plt.grid(True)
plt.savefig("loss_model2.jpg")
plt.close()

In [ ]:
import joblib
from os.path import join as join, exists
#Load model_b1
path = join('models', "model_20250618_1652_0_None.pkl")
if exists(path):
	model_b1_loaded = joblib.load(path)
	print(model_b1_loaded.layers)
else:
    print(f"❌ File not found: {join('..', 'models', 'model_20250618_1652_0_None.pkl')}")

# les colonnes ajoutés sont "nb_enfants", "quotient_caf"
# Il va falloir les ajouter à la main dans le modèle au bon endroit



[<Dense name=dense, built=True>, <Dense name=dense_1, built=True>, <Dense name=dense_2, built=True>]


In [2]:
for layer in model_b1_loaded.layers:
    try:
        # old_layer = model1_loaded.get_layer(layer.name)
        # layer.set_weights(old_layer.get_weights())
        print(f"✅ Poids layer : {layer.name}")
    except ValueError:
        print(f"⛔ Incompatible ou nouvelle couche : {layer.name}")

✅ Poids layer : dense
✅ Poids layer : dense_1
✅ Poids layer : dense_2
